Warning: Depending on which part of the song the classifier should be trained, different part of the code needs to be commented out. Right now, the script is focusing on a SVM for the end part of the songs.

### Import libraries and data

In [1]:
# for data manipulations
import numpy as np
import pandas as pd

# for PCA
from sklearn.decomposition import PCA

# for SVM
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import classification_report, multilabel_confusion_matrix

# save models
import joblib
from joblib import dump
import shutil

In [2]:
# # load relevant training data (start of song)
# X_train = np.load("X_train_start.npy")
# y_train = np.load("y_train_start.npy")

# # load relevant training data
# X_val = np.load("X_val_start.npy")
# y_val = np.load("y_val_start.npy")

# # load relevant test data
# X_test = np.load("X_test_start.npy")
# y_test = np.load("y_test_start.npy")

In [3]:
# # load relevant training data (middle of song)
# X_train = np.load("X_train_middle.npy")
# y_train = np.load("y_train_middle.npy")

# # load relevant training data
# X_val = np.load("X_val_middle.npy")
# y_val = np.load("y_val_middle.npy")

# # load relevant test data
# X_test = np.load("X_test_middle.npy")
# y_test = np.load("y_test_middle.npy")

In [4]:
# load relevant training data (end of song)
X_train = np.load("X_train_end.npy")
y_train = np.load("y_train_end.npy")

# load relevant training data
X_val = np.load("X_val_end.npy")
y_val = np.load("y_val_end.npy")

# load relevant test data
X_test = np.load("X_test_end.npy")
y_test = np.load("y_test_end.npy")

In [5]:
# check shape of X
print("Shape of X_train: ", X_train.shape, "\n")
print("Shape of X_val: ", X_val.shape, "\n")
print("Shape of X_test: ", X_test.shape, "\n")

# check shape of Y
print("Shape of y_train: ", y_train.shape, "\n")
print("Shape of y_val: ", y_val.shape, "\n")
print("Shape of y_test: ", y_test.shape, "\n")

Shape of X_train:  (4251, 1292, 20, 1) 

Shape of X_val:  (1418, 1292, 20, 1) 

Shape of X_test:  (1418, 1292, 20, 1) 

Shape of y_train:  (4251, 10) 

Shape of y_val:  (1418, 10) 

Shape of y_test:  (1418, 10) 



### Reshape features according to the model

In [6]:
# append validation data to training data because grid-search with cv is used
X_train = np.append(X_train, X_val, axis=0)
y_train = np.append(y_train, y_val, axis=0)

In [7]:
# check that append worked
print(X_train.shape)
print(y_train.shape)

(5669, 1292, 20, 1)
(5669, 10)


In [8]:
# remove channel dimension of MFCCs
# reshape features accordingly to SVM architecture
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [9]:
# check that removal of channel dimension worked
print(X_train.shape)
print(X_test.shape)

(5669, 25840)
(1418, 25840)


### PCA to reduce number of features

In [10]:
# perform PCA on X to reduce amount of features
# reduce to dimensionality so that 80% of variance is kept
pca = PCA(n_components=0.8, random_state=42)
X_pca_train = pca.fit_transform(X_train)

In [11]:
# check how many principal components are needed
pca.n_components_

461

In [12]:
# apply the trained PCA model on the test set as well
X_pca_test = pca.transform(X_test)

In [13]:
# change to original variable convention
X_train = X_pca_train
X_test = X_pca_test

In [14]:
# check that PCA worked
print(X_train.shape)
print(X_test.shape)

(5669, 461)
(1418, 461)


### Train linear SVM

Disclaimer: Linear SVM and SVM with RBF kernel were trained with grid-search 5-fold cross-validation separately because grid-search trains all combinations including gamma combinations for linear SVM which is not a relevant parameter that model (save unnecessary computations).

In [15]:
# define the linear SVM model
svm = SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42)
multi_svm = ClassifierChain(svm)

# define the hyperparameter grid
param_grid = {
    # the selection of the hyperparameters was in the beginning larger and reduced after some tries
    # restricting the model with smaller values made the best results
    'base_estimator__C':[2**-5, 2**-4, 2**-3, 2**-1],
}

# define the scoring metric
scorer = make_scorer(roc_auc_score, multi_class='ovr', average='macro')

# define the grid search
grid = GridSearchCV(
    multi_svm,
    param_grid,
    cv=5,
    scoring=scorer,
    refit=True,
    return_train_score=True,
    verbose=3)

In [16]:
# train the SVMs
grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END base_estimator__C=0.03125;, score=(train=0.693, test=0.564) total time= 2.0min
[CV 2/5] END base_estimator__C=0.03125;, score=(train=0.690, test=0.572) total time= 2.1min
[CV 3/5] END base_estimator__C=0.03125;, score=(train=0.686, test=0.555) total time= 2.0min
[CV 4/5] END base_estimator__C=0.03125;, score=(train=0.673, test=0.570) total time= 2.1min
[CV 5/5] END base_estimator__C=0.03125;, score=(train=0.690, test=0.567) total time= 2.1min
[CV 1/5] END base_estimator__C=0.0625;, score=(train=0.677, test=0.557) total time= 2.1min
[CV 2/5] END base_estimator__C=0.0625;, score=(train=0.694, test=0.568) total time= 2.0min
[CV 3/5] END base_estimator__C=0.0625;, score=(train=0.686, test=0.555) total time= 2.1min
[CV 4/5] END base_estimator__C=0.0625;, score=(train=0.674, test=0.566) total time= 2.0min
[CV 5/5] END base_estimator__C=0.0625;, score=(train=0.680, test=0.562) total time= 2.1min
[CV 1/5] END base_estimat

GridSearchCV(cv=5,
             estimator=ClassifierChain(base_estimator=SVC(class_weight='balanced',
                                                          kernel='linear',
                                                          probability=True,
                                                          random_state=42)),
             param_grid={'base_estimator__C': [0.03125, 0.0625, 0.125, 0.5]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, multi_class=ovr, average=macro),
             verbose=3)

In [17]:
# check performance of the models
pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_train_score"], columns=["Training Mean AUC"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["Validation Mean AUC"])], axis=1)

,base_estimator__C,Training Mean AUC,Validation Mean AUC
0,0.03125,0.686360,0.565581
1,0.06250,0.682178,0.561746
2,0.12500,0.676061,0.557531
3,0.50000,0.680477,0.553829


In [18]:
# print the best hyperparameters
print('Best Hyperparameters: ', grid.best_params_)

Best Hyperparameters:  {'base_estimator__C': 0.03125}


##### Option to save the model

In [19]:
# save the best model (start of song)
# dump(grid, 'svm_linear_start.joblib') # without zip
# shutil.make_archive('svm_linear_start', 'zip', '.', 'svm_linear_start.joblib') # with zip

In [20]:
# save the best model (middle of song)
# dump(grid, 'svm_linear_middle.joblib') # without zip
# shutil.make_archive('svm_linear_middle', 'zip', '.', 'svm_linear_middle.joblib') # with zip

In [21]:
# save the best model (end of song)
# dump(grid, 'svm_linear_end.joblib') # without zip
# shutil.make_archive('svm_linear_end', 'zip', '.', 'svm_linear_end.joblib') # with zip

'/work/end/svm_linear_end.zip'

##### Option to load the model if already trained

In [22]:
# load the model if already trained (start)
# grid = joblib.load('svm_linear_start.joblib')

In [23]:
# load the model if already trained (middle)
# grid = joblib.load('svm_linear_middle.joblib')

In [24]:
# load the model if already trained (end)
# grid = joblib.load('svm_linear_end.joblib')

##### Get scoring metrics on the test set

In [25]:
# make predictions on the test set (probabilities)
y_pred_prob = grid.predict_proba(X_test)

In [26]:
# check predictions (probabilities)
y_pred_prob[:5].round(2)

array([[0.04, 0.52, 0.19, 0.03, 0.1 , 0.05, 0.06, 0.03, 0.  , 0.12],
       [0.04, 0.22, 0.1 , 0.03, 0.06, 0.16, 0.02, 0.  , 0.  , 0.02],
       [0.06, 0.38, 0.11, 0.05, 0.09, 0.1 , 0.05, 0.01, 0.01, 0.05],
       [0.06, 0.06, 0.18, 0.02, 0.17, 0.04, 0.1 , 0.13, 0.08, 0.12],
       [0.07, 0.1 , 0.07, 0.03, 0.09, 0.11, 0.04, 0.51, 0.03, 0.11]])

In [27]:
# make predictions on the test set
y_pred = grid.predict(X_test)

In [28]:
# check predictions 
y_pred[:5]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [29]:
# get auc from the test set
auc_score = roc_auc_score(y_test, y_pred_prob, average='macro')

In [30]:
# print the results
print("AUC: :", auc_score)

AUC: : 0.6141884565134564


In [31]:
# print also the other metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.11      0.49      0.18        99
           1       0.36      0.70      0.48       266
           2       0.36      0.31      0.33       334
           3       0.04      0.17      0.06        46
           4       0.20      0.33      0.25       193
           5       0.20      0.10      0.14       202
           6       0.14      0.39      0.21       114
           7       0.32      0.40      0.36       252
           8       0.30      0.04      0.07       160
           9       0.15      0.39      0.22       152

   micro avg       0.22      0.35      0.27      1818
   macro avg       0.22      0.33      0.23      1818
weighted avg       0.26      0.35      0.27      1818
 samples avg       0.25      0.35      0.27      1818



In [33]:
confusion_matrix = multilabel_confusion_matrix(y_test, y_pred)
    
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

for i, matrix in enumerate(confusion_matrix):
    true_positives += matrix[1, 1]
    true_negatives += matrix[0, 0]
    false_positives += matrix[0, 1]
    false_negatives += matrix[1, 0]

multi_label_accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

print("Multi-label accuracy:", multi_label_accuracy)

Multi-label accuracy: 0.7578279266572637


### Train SVM with RBF kernel

Disclaimer: Linear SVM and SVM with RBF kernel were trained with grid-search 5-fold cross-validation separately because grid-search trains all combinations including gamma combinations for linear SVM which is not a relevant parameter that model (save unnecessary computations).

In [34]:
# define the SVM model with RBF kernel
svm = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
multi_svm = ClassifierChain(svm)

# define the hyperparameter grid
param_grid = {
    # the selection of the hyperparameters was in the beginning larger and reduced after some tries
    # restricting the model with smaller values made the best results
    'base_estimator__C':[2**-5, 2**-4, 2**-3],
    'base_estimator__gamma':[2**-11, 2**-9, 2**-7, 2**-5]
}

# define the scoring metric
scorer = make_scorer(roc_auc_score, multi_class='ovr', average='macro')

# define the grid search
grid = GridSearchCV(
    multi_svm,
    param_grid,
    cv=5,
    scoring=scorer,
    refit=True,
    return_train_score=True,
    verbose=3)

In [35]:
# train the SVMs
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END base_estimator__C=0.03125, base_estimator__gamma=0.00048828125;, score=(train=0.586, test=0.599) total time= 3.7min
[CV 2/5] END base_estimator__C=0.03125, base_estimator__gamma=0.00048828125;, score=(train=0.596, test=0.587) total time= 3.6min
[CV 3/5] END base_estimator__C=0.03125, base_estimator__gamma=0.00048828125;, score=(train=0.593, test=0.578) total time= 3.5min
[CV 4/5] END base_estimator__C=0.03125, base_estimator__gamma=0.00048828125;, score=(train=0.594, test=0.575) total time= 3.5min
[CV 5/5] END base_estimator__C=0.03125, base_estimator__gamma=0.00048828125;, score=(train=0.595, test=0.598) total time= 3.5min
[CV 1/5] END base_estimator__C=0.03125, base_estimator__gamma=0.001953125;, score=(train=0.629, test=0.634) total time= 3.3min
[CV 2/5] END base_estimator__C=0.03125, base_estimator__gamma=0.001953125;, score=(train=0.636, test=0.615) total time= 3.3min
[CV 3/5] END base_estimator__C=0.03125, 

GridSearchCV(cv=5,
             estimator=ClassifierChain(base_estimator=SVC(class_weight='balanced',
                                                          probability=True,
                                                          random_state=42)),
             param_grid={'base_estimator__C': [0.03125, 0.0625, 0.125],
                         'base_estimator__gamma': [0.00048828125, 0.001953125,
                                                   0.0078125, 0.03125]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, multi_class=ovr, average=macro),
             verbose=3)

In [37]:
# check the models
pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_train_score"], columns=["Training Mean AUC"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["Validation Mean AUC"])], axis=1)

,base_estimator__C,base_estimator__gamma,Training Mean AUC,Validation Mean AUC
0,0.03125,0.000488,0.592729,0.587331
1,0.03125,0.001953,0.632815,0.620491
2,0.03125,0.007812,0.659810,0.629262
3,0.03125,0.031250,0.633980,0.571620
4,0.06250,0.000488,0.617088,0.607488
5,0.06250,0.001953,0.647213,0.622991
6,0.06250,0.007812,0.686709,0.634739
7,0.06250,0.031250,0.693563,0.593053
8,0.12500,0.000488,0.633123,0.617958
9,0.12500,0.001953,0.665104,0.628820


In [38]:
# print the best hyperparameters
print('Best Hyperparameters: ', grid.best_params_)

Best Hyperparameters:  {'base_estimator__C': 0.125, 'base_estimator__gamma': 0.0078125}


##### Option to save the model

In [39]:
# save the best model (start of song)
# dump(grid, 'svm_rbf_start.joblib') # without zip
# shutil.make_archive('svm_rbf_start', 'zip', '.', 'svm_rbf_start.joblib') # with zip

In [40]:
# save the best model (middle of song)
# dump(grid, 'svm_rbf_middle.joblib') # without zip
# shutil.make_archive('svm_rbf_middle', 'zip', '.', 'svm_rbf_middle.joblib') # with zip

In [41]:
# save the best model (end of song)
# dump(grid, 'svm_rbf_end.joblib') # without zip
# shutil.make_archive('svm_rbf_end', 'zip', '.', 'svm_rbf_end.joblib') # with zip

'/work/end/svm_rbf_end.zip'

##### Option to load the model if already trained

In [42]:
# load the model if already trained (start of song)
# grid = joblib.load('svm_rbf_start.joblib')

In [43]:
# load the model if already trained (middle of song)
# grid = joblib.load('svm_rbf_middle.joblib')

In [44]:
# load the model if already trained (end of song)
# grid = joblib.load('svm_rbf_end.joblib')

##### Get scoring metrics on the test set

In [45]:
# make predictions on the test set (probabilities)
y_pred_prob = grid.predict_proba(X_test)

In [52]:
# check predictions (probabilities)
y_pred_prob[:5].round(2)

array([[0.01, 0.62, 0.37, 0.01, 0.07, 0.07, 0.12, 0.01, 0.01, 0.15],
       [0.07, 0.06, 0.15, 0.03, 0.07, 0.1 , 0.08, 0.3 , 0.14, 0.05],
       [0.02, 0.32, 0.18, 0.08, 0.06, 0.33, 0.04, 0.02, 0.04, 0.09],
       [0.2 , 0.03, 0.1 , 0.02, 0.19, 0.06, 0.04, 0.48, 0.13, 0.03],
       [0.09, 0.05, 0.06, 0.06, 0.07, 0.18, 0.02, 0.44, 0.18, 0.03]])

In [46]:
# make predictions on the test set
y_pred = grid.predict(X_test)

In [47]:
# check predictions 
y_pred[:5]

array([[0., 1., 1., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0., 1., 0., 1., 1., 0.]])

In [48]:
# get auc from the test set
auc_score = roc_auc_score(y_test, y_pred_prob, average='macro')

In [49]:
# print the results
print("AUC: :", auc_score)

AUC: : 0.7086709248166961


In [50]:
# print also the other metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.11      0.65      0.19        99
           1       0.36      0.77      0.49       266
           2       0.38      0.66      0.48       334
           3       0.05      0.30      0.08        46
           4       0.21      0.49      0.30       193
           5       0.27      0.48      0.34       202
           6       0.16      0.55      0.24       114
           7       0.30      0.74      0.43       252
           8       0.22      0.62      0.32       160
           9       0.18      0.64      0.28       152

   micro avg       0.24      0.63      0.34      1818
   macro avg       0.22      0.59      0.32      1818
weighted avg       0.27      0.63      0.37      1818
 samples avg       0.25      0.64      0.34      1818



In [51]:
confusion_matrix = multilabel_confusion_matrix(y_test, y_pred)
    
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

for i, matrix in enumerate(confusion_matrix):
    true_positives += matrix[1, 1]
    true_negatives += matrix[0, 0]
    false_positives += matrix[0, 1]
    false_negatives += matrix[1, 0]

multi_label_accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

print("Multi-label accuracy:", multi_label_accuracy)

Multi-label accuracy: 0.6909026798307475
